In [ ]:
!pip install ultralytics --quiet

INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 129.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.4/810.4 kB 28.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 136.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [ultralytics] [ultralytics]thop]

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Cell 2: Register PENetWrapper
from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules

_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper
print("✅ PENetWrapper registered in ultralytics.")

✅ PENetWrapper registered in ultralytics.


In [2]:
# ─── Build PENet+YOLOv8n, load pretrained, SAVE, then validate ───
import torch, copy
from ultralytics import YOLO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 1) Build model from custom YAML (PENet at position 0)
model = YOLO("yolov8-penet.yaml")
model.model.to(device)

# 2) Load pretrained YOLOv8n weights with +1 index shift
PRETRAINED_PATH = "runs/detect/train12/weights/best.pt"
pretrained = torch.load(PRETRAINED_PATH, weights_only=False, map_location=device)
pretrained_sd = pretrained["model"].state_dict()
model_sd = model.model.state_dict()

shifted = {}
for key, val in pretrained_sd.items():
    if key.startswith("model."):
        parts = key.split(".")
        if parts[1].isdigit():
            new_key = f"model.{int(parts[1]) + 1}.{'.'.join(parts[2:])}"
            if new_key in model_sd and model_sd[new_key].shape == val.shape:
                shifted[new_key] = val

model.model.load_state_dict(shifted, strict=False)
print(f"✅ Loaded {len(shifted)} pretrained weights (shifted +1 for PENet)")

# 3) SAVE BEFORE val() — val() fuses Conv+BN, destroying BN weights
INIT_CKPT = "penet_yolov8n_init.pt"
torch.save({
    "model": copy.deepcopy(model.model).half(),  # deepcopy = safe unfused snapshot
    "train_args": {
        "task": "detect",
        "data": "exdark.yaml",
        "imgsz": 640,
        "model": "yolov8-penet.yaml",  # needed so trainer knows the architecture
    },
}, INIT_CKPT)
print(f"✅ Saved UNFUSED model to {INIT_CKPT}")

# 4) Now validate (fuses model in-place, but we already saved the unfused copy)
print("\n── Validation to confirm pretrained weights loaded correctly ──")
metrics = model.val(data="exdark.yaml", imgsz=640, batch=64, workers=16)
print(f"mAP50 (should be ~0.65): {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

Device: cuda
WARNING ⚠️ no model scale passed. Assuming scale='n'.
✅ Loaded 355 pretrained weights (shifted +1 for PENet)
✅ Saved UNFUSED model to penet_yolov8n_init.pt

── Validation to confirm pretrained weights loaded correctly ──
Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
YOLOv8-penet summary (fused): 170 layers, 3,099,117 parameters, 105,949 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1189.5±518.3 MB/s, size: 131.7 KB)
val: Scanning /teamspace/studios/this_studio/labels/val.cache... 1799 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 1799/1799 269.5Mit/s 0.0s
val: /teamspace/studios/this_studio/images/val/2015_05337.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.075]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 29/29 1.2s/it 35.0s1.2ss
                   all       1798       6204      0.744      0.589      0.669      

In [3]:
# Cell 4: PHASE 1 — Train PENet only (YOLO frozen)
# Load from the checkpoint we just saved (preserves pretrained weights!)

from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

from ultralytics import YOLO

model = YOLO("penet_yolov8n_init.pt")

# Freeze everything except PENet (model.0.*)
for name, param in model.model.named_parameters():
    if not name.startswith("model.0"):
        param.requires_grad = False

trainable = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.model.parameters())
print(f"Phase 1 — trainable: {trainable:,} / {total:,} params")

results_p1 = model.train(
    data="exdark.yaml",
    epochs=20,
    imgsz=640,
    batch=48,
    lr0=1e-3,
    lrf=0.1,
    cos_lr=True,
    warmup_epochs=2,
    optimizer="AdamW",
    weight_decay=0.01,
    mosaic=0.5,
    mixup=0.0,
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.3,
    workers=16,
    cache="ram",
    amp=True,
    pretrained=False,
    project="runs/penet",
    name="phase1",
)

Phase 1 — trainable: 91,129 / 3,104,317 params
Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=exdark.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=penet_yolov8n_init.pt, momentum=0.937, mosaic=0.5, multi_scale=0.0, name=phase12, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, over

YOLOv8-penet summary: 227 layers, 3,104,317 parameters, 3,104,301 gradients

Transferred 448/448 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2523.7±735.6 MB/s, size: 119.7 KB)
train: Scanning /teamspace/studios/this_studio/labels/train.cache... 3000 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3000/3000 1.0Git/s 0.0s
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.
train: Caching images (2.5GB RAM): 100% ━━━━━━━━━━━━ 3000/3000 874.6it/s 3.4s0.2s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 332.0±151.7 MB/s, size: 75.2 KB)
val: Scanning /teamspace/studios/this_studio/labels/val.cache... 1799 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 1799/1799 76.2Mit/s 0.0s
val: /teamspace/studios/this_studio/images/val/2015_053

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/20      18.9G      1.133     0.9175      1.187        142        640: 100% ━━━━━━━━━━━━ 63/63 3.7s/it 3:513.8ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 34.9s2.0ss
                   all       1798       6204      0.507      0.365       0.37      0.213

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/20      17.4G       1.29      1.109      1.212        316        640: 0% ──────────── 0/63  3.6s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

      18/20        19G      1.036     0.6928      1.127         93        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 35.0s2.0ss
                   all       1798       6204      0.719      0.578      0.646      0.393

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      19/20        19G      1.031     0.6531        1.1        114        640: 0% ──────────── 0/63  3.6s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      19/20        19G      1.017     0.6696      1.122         99        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 34.9s2.0ss
                   all       1798       6204      0.715      0.582      0.652      0.401

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      20/20        19G     0.9766     0.6606      1.065        165        640: 0% ──────────── 0/63  3.7s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      20/20        19G     0.9992     0.6634      1.106         76        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 35.0s2.0ss
                   all       1798       6204      0.729      0.566      0.647      0.398

20 epochs completed in 1.454 hours.
Optimizer stripped from /teamspace/studios/this_studio/runs/detect/runs/penet/phase12/weights/last.pt, 6.5MB
Optimizer stripped from /teamspace/studios/this_studio/runs/detect/runs/penet/phase12/weights/best.pt, 6.5MB

Validating /teamspace/studios/this_studio/runs/detect/runs/penet/phase12/weights/best.pt...
Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
YOLOv8-penet summary (fused): 170 layers, 3,099,117 parameters, 0 gradients
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.9s/it 36.5s2.0ss
                 

In [7]:
# Quick debug
from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

from ultralytics import YOLO

model = YOLO("runs/detect/runs/penet/phase12/weights/best.pt")
penet_wrapper = model.model.model[0]

# See what's inside
print(type(penet_wrapper))
print(list(penet_wrapper._modules.keys()))
print(list(penet_wrapper._parameters.keys()))

<class 'PENet.PENetWrapper'>
['model']
[]


In [9]:
print(dir(penet_wrapper))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hoo

In [10]:
# Run this quick diagnostic cell after Phase 1

from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

import torch
from ultralytics import YOLO

model = YOLO("runs/detect/runs/penet/phase12/weights/best.pt")
penet_wrapper = model.model.model[0]
penet = penet_wrapper.model

# 1) Print PENet structure
print("PENet type:", type(penet))
print("PENet children:")
for name, module in penet.named_children():
    print(f"  {name}: {type(module).__name__}")

# 2) Check AE weights
print("\n--- AE weight statistics ---")
for name, param in penet.named_parameters():
    if 'res1' in name and 'weight' in name and 'block.0' in name:
        print(f"{name} — mean: {param.mean():.6f}, std: {param.std():.6f}, max: {param.abs().max():.6f}")

# 3) Visual test — does PENet actually change the image?
device = next(model.model.parameters()).device
dummy = torch.rand(1, 3, 640, 640, device=device)
with torch.no_grad():
    enhanced = penet_wrapper(dummy)

diff = (enhanced - dummy).abs()
print(f"\n--- PENet effect on random input ---")
print(f"Input  — mean: {dummy.mean():.6f}, std: {dummy.std():.6f}")
print(f"Output — mean: {enhanced.mean():.6f}, std: {enhanced.std():.6f}")
print(f"Mean pixel change:  {diff.mean():.6f}")
print(f"Max pixel change:   {diff.max():.6f}")
print(f"% pixels changed > 0.01: {(diff > 0.01).float().mean() * 100:.1f}%")

if diff.mean() < 1e-4:
    print("\n⚠️  PENet is effectively identity")
    print("   Phase 1 didn't learn useful enhancement")
    print("   Phase 2 will still work — YOLO gradients will force PENet to learn")
else:
    print(f"\n✅ PENet IS modifying images (mean change: {diff.mean():.4f})")
    print("   Phase 1 learned meaningful enhancement")

# 4) Compare with actual dark image from dataset
print("\n--- PENet total parameters ---")
total = sum(p.numel() for p in penet.parameters())
trainable = sum(p.numel() for p in penet.parameters() if p.requires_grad)
print(f"Total: {total:,} | Trainable: {trainable:,}")

PENet type: <class 'PENet.PENet'>
PENet children:
  lap_pyramid: Lap_Pyramid_Conv
  AEs: ModuleList

--- AE weight statistics ---
AEs.0.res1.block.0.weight — mean: 0.000128, std: 0.108371, max: 0.210083
AEs.1.res1.block.0.weight — mean: -0.013518, std: 0.114227, max: 0.247681
AEs.2.res1.block.0.weight — mean: -0.008836, std: 0.106599, max: 0.210938
AEs.3.res1.block.0.weight — mean: -0.009488, std: 0.113794, max: 0.214355

--- PENet effect on random input ---
Input  — mean: 0.499959, std: 0.288887
Output — mean: 0.505422, std: 0.289791
Mean pixel change:  0.006459
Max pixel change:   0.017066
% pixels changed > 0.01: 33.2%

✅ PENet IS modifying images (mean change: 0.0065)
   Phase 1 learned meaningful enhancement

--- PENet total parameters ---
Total: 91,129 | Trainable: 0


In [12]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO

model = YOLO("runs/detect/runs/penet/phase12/weights/best.pt")
penet_wrapper = model.model.model[0]
device = next(model.model.parameters()).device

# Get some dark images from the dataset
img_dir = Path("/teamspace/studios/this_studio/images/val")
img_paths = sorted(img_dir.glob("*.jpg")) + sorted(img_dir.glob("*.jpeg")) + sorted(img_dir.glob("*.png"))
sample_paths = img_paths[:6]  # take 6 samples

fig, axes = plt.subplots(len(sample_paths), 3, figsize=(15, 5 * len(sample_paths)))

for idx, img_path in enumerate(sample_paths):
    # Load and preprocess (same as YOLO pipeline)
    img_bgr = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    # Resize to 640x640
    img_resized = cv2.resize(img_rgb, (640, 640))
    
    # Normalize to [0, 1] and convert to tensor [1, 3, H, W]
    img_tensor = torch.from_numpy(img_resized).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    img_tensor = img_tensor.to(device)
    
    # Run through PENet
    with torch.no_grad():
        enhanced_tensor = penet_wrapper(img_tensor)
    
    # Compute difference
    diff_tensor = (enhanced_tensor - img_tensor).abs()
    
    # Convert back to numpy for display
    original = img_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    enhanced = enhanced_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy().clip(0, 1)
    diff = diff_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    
    # Scale diff for visibility (multiply by 20x)
    diff_vis = (diff * 20).clip(0, 1)
    
    # Plot
    axes[idx, 0].imshow(original)
    axes[idx, 0].set_title(f"Original: {img_path.name}", fontsize=10)
    axes[idx, 0].axis("off")
    
    axes[idx, 1].imshow(enhanced)
    axes[idx, 1].set_title("PENet Enhanced", fontsize=10)
    axes[idx, 1].axis("off")
    
    axes[idx, 2].imshow(diff_vis)
    axes[idx, 2].set_title(f"Difference (20x) | mean: {diff.mean():.4f}", fontsize=10)
    axes[idx, 2].axis("off")

plt.suptitle("PENet Enhancement After Phase 1", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.savefig("penet_phase1_visualization.png", dpi=150, bbox_inches="tight")
plt.show()
print("✅ Saved to penet_phase1_visualization.png")

<Figure size 1500x0 with 0 Axes>

<Figure size 1500x3000 with 18 Axes>

✅ Saved to penet_phase1_visualization.png


In [13]:
# Cell 5: PHASE 2 — Full end-to-end fine-tuning (SPEED OPTIMIZED for L4)

from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

import torch
from ultralytics import YOLO

# ── Speed optimizations for L4 (Ampere) ──
torch.backends.cudnn.benchmark = True        # auto-tune conv algorithms
torch.backends.cuda.matmul.allow_tf32 = True # TF32: 62.5 TFLOPS vs 30.3 FP32 (2x!)
torch.backends.cudnn.allow_tf32 = True       # TF32 for cuDNN convolutions

model = YOLO("runs/detect/runs/penet/phase12/weights/best.pt")

# Compile PENet for kernel fusion
if hasattr(model.model.model[0], 'penet'):
    model.model.model[0].penet = torch.compile(model.model.model[0].penet, mode="reduce-overhead")
    print("✅ PENet compiled with torch.compile")

for param in model.model.parameters():
    param.requires_grad = True

trainable = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
print(f"Phase 2 — all {trainable:,} params trainable")

results_p2 = model.train(
    data="exdark.yaml",
    epochs=80,
    imgsz=640,
    batch=48,              # keep 48 — OOM at 64 during val
    lr0=5e-4,
    lrf=0.01,
    cos_lr=True,
    warmup_epochs=3,
    optimizer="AdamW",
    weight_decay=0.005,
    mosaic=0.5,
    mixup=0.1,
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=5.0,
    translate=0.1,
    scale=0.3,
    workers=8,             # ↓ from 16 — you only have 8 vCPUs
    cache="ram",
    amp=True,              # FP16 — critical for L4
    pretrained=False,
    patience=20,
    project="runs/penet",
    name="phase2",
)

Phase 2 — all 3,104,317 params trainable
Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=exdark.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=runs/detect/runs/penet/phase12/weights/best.pt, momentum=0.937, mosaic=0.5, multi_scale=0.0, name=phase2, nbs=64, nms=False, opset=None, optimize=False, o

YOLOv8-penet summary: 227 layers, 3,104,317 parameters, 3,104,301 gradients

Transferred 448/448 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2197.2±701.4 MB/s, size: 119.7 KB)
train: Scanning /teamspace/studios/this_studio/labels/train.cache... 3000 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3000/3000 629.1Mit/s 0.0s
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.
train: Caching images (2.5GB RAM): 100% ━━━━━━━━━━━━ 3000/3000 960.7it/s 3.1s0.2s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 352.1±102.9 MB/s, size: 75.2 KB)
val: Scanning /teamspace/studios/this_studio/labels/val.cache... 1799 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 1799/1799 85.7Mit/s 0.0s
val: /teamspace/studios/this_studio/images/val/2015_0

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/80        19G      1.271      1.129      1.328        126        640: 100% ━━━━━━━━━━━━ 63/63 3.5s/it 3:432.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 35.0s2.0ss
                   all       1798       6204      0.578      0.421      0.445      0.225

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/80      18.9G      1.239      1.162      1.351        264        640: 0% ──────────── 0/63  3.6s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

      78/80      18.9G     0.7737     0.4432     0.9846         50        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 34.8s2.0ss
                   all       1798       6204      0.731      0.578      0.653      0.398

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      79/80      18.9G     0.7291      0.413     0.9924        141        640: 0% ──────────── 0/63  3.6s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      79/80      18.9G     0.7709     0.4393     0.9835         86        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 34.7s2.0ss
                   all       1798       6204      0.742      0.572      0.653      0.399

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      80/80      18.9G     0.9182     0.4925      1.021        165        640: 0% ──────────── 0/63  3.6s

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/autograd/graph.py:829: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      80/80      18.9G     0.7631     0.4385     0.9804         67        640: 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:452.9ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.8s/it 34.7s2.0ss
                   all       1798       6204      0.736      0.577      0.653      0.398

80 epochs completed in 5.805 hours.
Optimizer stripped from /teamspace/studios/this_studio/runs/detect/runs/penet/phase2/weights/last.pt, 6.5MB
Optimizer stripped from /teamspace/studios/this_studio/runs/detect/runs/penet/phase2/weights/best.pt, 6.5MB

Validating /teamspace/studios/this_studio/runs/detect/runs/penet/phase2/weights/best.pt...
Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
YOLOv8-penet summary (fused): 170 layers, 3,099,117 parameters, 0 gradients
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.9s/it 36.1s2.0ss
                   a

In [ ]:
# Cell 6: Evaluate final model

from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

from ultralytics import YOLO

final = YOLO("runs/detect/runs/penet/phase2/weights/best.pt")
metrics = final.val(data="exdark.yaml", imgsz=640, batch=64, workers=16)
print(f"\nmAP50:    {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

Ultralytics 8.4.14 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22813MiB)
Model summary (fused): 73 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1663.8±635.1 MB/s, size: 93.9 KB)
val: Scanning /teamspace/studios/this_studio/labels/val.cache... 1799 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 1799/1799 419.2Mit/s 0.0s
val: /teamspace/studios/this_studio/images/val/2015_05337.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.075]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 113/113 8.9it/s 12.6s<0.1s
                   all       1798       6204      0.742      0.587      0.668      0.419
               Bicycle        170        237      0.772      0.727      0.781      0.502
                  Boat        152        344      0.711      0.506      0.591      0.296
                Bottle        211        504      0

In [ ]:
# Cell 7: (Optional) Phase 3 — Aggressive fine-tuning

from PENet import PENetWrapper
import ultralytics.nn.tasks as _tasks
import ultralytics.nn.modules as _modules
_tasks.__dict__["PENetWrapper"] = PENetWrapper
_modules.__dict__["PENetWrapper"] = PENetWrapper

from ultralytics import YOLO

model = YOLO("runs/detect/runs/penet/phase2/weights/best.pt")
for param in model.model.parameters():
    param.requires_grad = True

results_p3 = model.train(
    data="exdark.yaml",
    epochs=40,
    imgsz=640,
    batch=48,
    lr0=1e-4,
    lrf=0.01,
    cos_lr=True,
    warmup_epochs=2,
    optimizer="AdamW",
    weight_decay=0.001,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.1,
    hsv_h=0.02,
    hsv_s=0.5,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.15,
    scale=0.5,
    workers=16,
    cache="ram",
    amp=True,
    pretrained=False,
    patience=15,
    project="runs/penet",
    name="phase3",
)